In [1]:
from rdflib.plugins.sparql import parser

In [379]:
# query = "SELECT ?x WHERE { ?x :p1/:p2 ?z ; !:p3 ?a }"
query = \
"PREFIX : <http://example.org/gmark/> " + \
"SELECT DISTINCT ?x0 ?x3 WHERE {  { " + \
"?x0 (((^:p1/:p1)|(^:p1/:p1/^:p1/:p1))) ?x1 . ?x1 (((^:p1/:p1/^:p1/:p1)|(^:p1/:p1))) ?x2 . ?x2 ((^:p1/:p1)|(^:p1/:p1/^:p1/:p1)) ?x3 . " + \
"} }"


In [380]:
query = parser.parseQuery(query)
if len(query) > 1:
    query = query[1]

In [381]:
query

SelectQuery_{'modifier': 'DISTINCT', 'projection': [vars_{'var': rdflib.term.Variable('x0')}, vars_{'var': rdflib.term.Variable('x3')}], 'where': GroupGraphPatternSub_{'part': [GroupOrUnionGraphPattern_{'graph': [GroupGraphPatternSub_{'part': [TriplesBlock_{'triples': [ParseResults([rdflib.term.Variable('x0'), PathAlternative_{'part': [PathSequence_{'part': [PathElt_{'part': PathAlternative_{'part': [PathSequence_{'part': [PathElt_{'part': PathAlternative_{'part': [PathSequence_{'part': [PathElt_{'part': PathAlternative_{'part': [PathSequence_{'part': [PathEltOrInverse_{'part': PathElt_{'part': pname_{'localname': 'p1'}}}, PathElt_{'part': pname_{'localname': 'p1'}}]}]}}]}, PathSequence_{'part': [PathElt_{'part': PathAlternative_{'part': [PathSequence_{'part': [PathEltOrInverse_{'part': PathElt_{'part': pname_{'localname': 'p1'}}}, PathElt_{'part': pname_{'localname': 'p1'}}, PathEltOrInverse_{'part': PathElt_{'part': pname_{'localname': 'p1'}}}, PathElt_{'part': pname_{'localname': 'p

In [397]:
class Print_Visitor:
    
    def visit(self, ast, lvl=0):
        if type(ast) == parser.ParseResults:
            self.results(ast, lvl)
            for j in ast.as_list(): self.visit(j, lvl+1)
        
        elif type(ast) == list:
            for j in ast: self.visit(j, lvl+1)
       
        elif hasattr(ast, 'items') and len(ast) > 0:
            self.node(ast, lvl)
            for j in ast.values(): self.visit(j, lvl+1)
        
        else:
            return self.leaf(ast, lvl)
    
    def __indent(self, lvl):
        return " ".join([ "" for _ in range(lvl) ])
    
    def results(self, ast, lvl):
        print(self.__indent(lvl), "results:", ast)
        
    def node(self, ast, lvl):
        print(self.__indent(lvl), "node:", ast.name)
    
    def leaf(self, ast, lvl):
        print(self.__indent(lvl), "leaf:", ast)

In [398]:
Print_Visitor().visit(query)

 node: SelectQuery
 leaf: DISTINCT
  node: vars
   leaf: x0
  node: vars
   leaf: x3
 node: GroupGraphPatternSub
   node: GroupOrUnionGraphPattern
     node: GroupGraphPatternSub
       node: TriplesBlock
         results: [rdflib.term.Variable('x0'), PathAlternative_{'part': [PathSequence_{'part': [PathElt_{'part': PathAlternative_{'part': [PathSequence_{'part': [PathElt_{'part': PathAlternative_{'part': [PathSequence_{'part': [PathElt_{'part': PathAlternative_{'part': [PathSequence_{'part': [PathEltOrInverse_{'part': PathElt_{'part': pname_{'localname': 'p1'}}}, PathElt_{'part': pname_{'localname': 'p1'}}]}]}}]}, PathSequence_{'part': [PathElt_{'part': PathAlternative_{'part': [PathSequence_{'part': [PathEltOrInverse_{'part': PathElt_{'part': pname_{'localname': 'p1'}}}, PathElt_{'part': pname_{'localname': 'p1'}}, PathEltOrInverse_{'part': PathElt_{'part': pname_{'localname': 'p1'}}}, PathElt_{'part': pname_{'localname': 'p1'}}]}]}}]}]}}]}]}}]}]}, rdflib.term.Variable('x1')]
       

In [395]:
class To_N3_Visitor:
    
    def visit(self, ast):
        if type(ast) == parser.ParseResults:
            ret = [ self.visit(j) for j in ast.as_list() ]
            return " . ".join([ " ".join(ret[n*3 : (n+1)*3]) for n in range(int(len(ret)/3)) ])
                    
        elif type(ast) == list:
            return " ".join([ self.visit(j) for j in ast ])
       
        elif hasattr(ast, 'name'):
            if ast.name == 'pname':
                return self.leaf(ast)            
            elif ast.name == 'TriplesBlock':
                ret = [ self.visit(j) for j in ast['triples'] ]
                return " . ".join(ret)
            else:
                return self.node(ast)
        
        else:
            return self.leaf(ast)
        
    def node(self, ast):        
        sep = None; 
        match(ast.name):
            case 'PathAlternative':
                sep = " '|' "
            case 'PathSequence':
                sep = " '/' "
            case 'PathNegatedPropertySet':
                sep = "'^' "
            case 'PathEltOrInverse':
                sep = "'^ "
            # (no need for separator for PathElt)
        
        key = list(ast.keys())[0]
        is_list = (type(ast[key]) == list)
        if is_list:
            nodes = [ self.visit(j) for j in ast[key] ]
        else:
            nodes = [ self.visit(ast[key]) ]

        if sep is not None:
            if is_list:
                return "(" + sep.join(nodes) + ")" if len(nodes) > 1 else sep.join(nodes)
            else:
                return "(" + sep + nodes[0] + ")"
        else:
            return nodes[0]
        
    def leaf(self, ast):
        if hasattr(ast, "n3") and ast.n3 is not None:
            return ast.n3()
        elif ast.name == "pname":
            return ":" + ast['localname']
        else:
            return ast.__str__()

In [396]:
visitor = To_N3_Visitor()
visitor.visit(query['where'])

"?x0 ((('^ :p1) '/' :p1) '|' (('^ :p1) '/' :p1 '/' ('^ :p1) '/' :p1)) ?x1 . ?x1 ((('^ :p1) '/' :p1 '/' ('^ :p1) '/' :p1) '|' (('^ :p1) '/' :p1)) ?x2 . ?x2 ((('^ :p1) '/' :p1) '|' (('^ :p1) '/' :p1 '/' ('^ :p1) '/' :p1)) ?x3"